## Imports

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

# Load Data

In [6]:
with open('./data/train.txt', 'r', encoding="utf8") as f:
    text = f.read()

# Pre processing step

### Tokenization 

In [24]:
def char_to_int(text):
    chars = tuple(set(text))
    int2char = dict(enumerate(chars))
    char2int = {ch: ii for ii, ch in int2char.items()}
    
    encoded = np.array([char2int[ch] for ch in text])
    return encoded

### One hot encoding

In [25]:
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [27]:
tokinized_text = char_to_int(text)

# Batching

# Building Model